# Part I : From a raw dataset to a meaningful one

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [167]:
df = pd.read_excel('sbf120_as_of_end_2018.xlsx', sheet_name=None, engine='openpyxl')

In [168]:
dataset = df['Data'].copy()
companies = df['Compo 31122018'].copy()

dfs = []

# Loop through the 120 sets of 3 columns (each set has a date, price, and an extra column)
for i in range(0, 360, 3):
    # Extract the relevant date and price columns
    date_col = pd.to_datetime(dataset.iloc[:,i].astype(float), origin='1899-12-30', unit='D')  # Date column
    price_cap_col = dataset.iloc[:, i+1:i+3]  # Price and Market cap column
    
    # Create a temporary DataFrame with these two columns
    temp_df = pd.DataFrame({
        'Date': pd.to_datetime(date_col, errors='coerce'),  
        f'{dataset.columns[i]}': price_cap_col.iloc[:, 0],  # Rename the price column to indicate the set number
        f' Market Cap {dataset.columns[i]}' : price_cap_col.iloc[:, 1] # Rename the Market Cap column to indicate the market cap
    })
    temp_df.set_index('Date', inplace=True) # set_index done in class to make a join and not a merge
    temp_df.dropna(inplace=True) # added to clean
    # Append the DataFrame to the list
    dfs.append(temp_df)

result = dfs[0]
for i in range(1,120):
    result = result.join(dfs[i], how='outer')
    
result

,QDT FP Equity,Market Cap QDT FP Equity,DIM FP Equity,Market Cap DIM FP Equity,RUI FP Equity,Market Cap RUI FP Equity,OR FP Equity,Market Cap OR FP Equity,DG FP Equity,Market Cap DG FP Equity,...,RCO FP Equity,Market Cap RCO FP Equity,STM FP Equity,Market Cap STM FP Equity,ATE FP Equity,Market Cap ATE FP Equity,WLN FP Equity,Market Cap WLN FP Equity,NXI FP Equity,Market Cap NXI FP Equity
Date,,,,,,,,,,,,,,,,,,,,,
2010-12-31,65.20,2074.1424,6.28,640.6889,21.1225,1224.8148,83.08,49784.4831,40.2799,22463.6902,...,51.1749,2603.4359,7.739,7045.2320,24.585,785.1392,NaN,NaN,26.4957,1770.6669
2011-01-03,65.41,2080.8229,6.29,641.5386,21.3261,1236.6203,85.50,51234.6330,41.3097,23037.9831,...,51.4648,2618.1862,8.081,7356.5732,24.700,788.8118,NaN,NaN,26.8603,1795.0327
2011-01-04,66.54,2116.7705,6.28,640.6889,21.3770,1239.5716,85.04,50958.9847,41.0176,22875.0827,...,51.2522,2607.3693,8.096,7370.2285,25.090,801.2667,NaN,NaN,27.2288,1819.6576
2011-01-05,66.51,2115.8161,6.29,641.3687,21.6896,1257.7014,84.66,50731.2752,40.4730,22571.3701,...,52.0833,2649.6536,8.287,7544.1062,25.000,798.3925,NaN,NaN,27.1047,1811.3629
2011-01-06,67.56,2149.2187,6.30,642.3883,21.6920,1257.8420,84.57,50677.3440,39.9878,22317.5758,...,51.3198,2610.8110,8.406,7652.4383,25.450,812.7636,NaN,NaN,27.1861,1816.8063
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-01,23.72,819.8323,524.20,48320.8556,33.2700,3459.1215,399.50,222789.8633,93.7500,56029.0137,...,170.0000,8589.5157,37.920,34554.1988,138.700,4765.8506,75.99,21312.5103,43.5800,2446.1334
2021-09-02,23.76,821.2148,532.60,49095.1692,33.2000,3451.8435,400.15,223152.3500,92.9800,55568.8288,...,170.9000,8634.9896,37.935,34567.8674,139.900,4807.0836,75.01,21037.6549,43.3600,2433.7848
2021-09-03,23.28,804.6246,531.00,48947.6809,32.9700,3427.9301,394.85,220196.6897,91.1400,54469.1660,...,168.6000,8518.7785,37.900,34535.9740,139.800,4803.6475,72.39,20302.8375,43.6600,2450.6237


In [169]:
for col in result.columns:
    # Find the first and last non-NaN index for each column
    first_valid_idx = result[col].first_valid_index()
    last_valid_idx = result[col].last_valid_index()
    
    # If there are valid indices (i.e., the column is not entirely NaN)
    if first_valid_idx is not None and last_valid_idx is not None:
        # Apply forward fill only between the first and last valid indices
        result.loc[first_valid_idx:last_valid_idx, col] = result.loc[first_valid_idx:last_valid_idx, col].bfill()

### Figure 1 : Prices of BNP FP & GLE FP

In [170]:
%matplotlib
df=result

# Filter the data to start from 2019
df = df[df.index >= '2019-01-01']

# Create a plot with two y-axes
fig, ax1 = plt.subplots(figsize=(8, 5))

# Plotting the first time series with green color
ax1.plot(df.index, df['BNP FP Equity'], color='green', label='BNP FP Equity')
ax1.set_ylabel('BNP FP Equity')
ax1.tick_params(axis='y')

# Create a second y-axis
ax2 = ax1.twinx()
ax2.plot(df.index, df['GLE FP Equity'], color='red', label='GLE FP Equity')
ax2.set_ylabel('GLE FP Equity')
ax2.tick_params(axis='y')

# Title and legend
ax1.set_title('Stock Prices Comparison Over Time')
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

# Add a vertical grid with dashed lines
ax1.grid(True, which='major', axis='x', linestyle='--')

# Annotating the graph for February 2020 (COVID)
annotation_date = pd.Timestamp('2020-02-28')
ax2.annotate('COVID',
             xy=(annotation_date, df.loc[annotation_date, 'GLE FP Equity']),
             xytext=(annotation_date, df['GLE FP Equity'].mean()+5),
             arrowprops=dict(facecolor='black', arrowstyle='->'),
             fontsize=12)

# Show the plot
plt.tight_layout()
plt.show()

Using matplotlib backend: Qt5Agg


### Figure 2 : One-dimensional analysis

In [171]:
import seaborn as sns


df = result[['BNP FP Equity', 'GLE FP Equity']].copy()
# Filter the data to start from 2019
df = df[df.index >= '2019-01-01'].pct_change()
df['Date'] = df.index
df['Year'] = df['Date'].dt.year

# Melt the DataFrame to long format
df_melted = df.melt(id_vars=['Date', 'Year'], var_name='Asset', value_name='Return')

# Plot the violin plot
plt.figure(figsize=(10, 5))
sns.violinplot(x='Year', y='Return', hue='Asset', data=df_melted, split=True)
plt.title('Distribution of daily price returns')
plt.ylabel('daily price return')
plt.xlabel('Year')
plt.legend(title='')
plt.tight_layout()
plt.show()

### Figure 3 : Two dimensionnal analysis

In [172]:

# Step 3: Plot with sns.jointplot using 'hue' for year
sns.jointplot(
    data=df,
    x='BNP FP Equity',
    y='GLE FP Equity',
    hue='Year',
    kind='scatter',
    palette='colorblind',  # or 'rocket', 'coolwarm', etc.
    height=7
)

plt.suptitle("Joint distribution of daily price returns")
plt.tight_layout()
plt.show()


# Part II : A class for analysing strategies

In [173]:
#Let's use the dataframe already built in the first part in order to
#lead the calculations of different Strategies.

daily_returns = result[(result.index>='2019-01-01')*(result.index<'2020-01-01')].iloc[:,::2].pct_change()
daily_market_cap = result[(result.index>='2019-01-01')*(result.index<'2020-01-01')].iloc[:,1::2]

In [174]:
daily_returns

,QDT FP Equity,DIM FP Equity,RUI FP Equity,OR FP Equity,DG FP Equity,ALD FP Equity,URW NA Equity,KORI FP Equity,TTE FP Equity,DSY FP Equity,...,ATO FP Equity,MDM FP Equity,AKE FP Equity,MERY FP Equity,RI FP Equity,RCO FP Equity,STM FP Equity,ATE FP Equity,WLN FP Equity,NXI FP Equity
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-03,0.033613,-0.030268,-0.021776,-0.012024,-0.003359,-0.009756,0.005127,0.003310,-0.008574,-0.063879,...,-0.052940,0.010657,-0.009680,0.004143,-0.015935,-0.031186,-0.116097,-0.030366,-0.062951,-0.005198
2019-01-04,-0.021138,0.033013,0.026541,0.014199,0.019382,0.055172,0.036159,0.018468,0.024970,0.039157,...,0.036775,-0.033392,0.057019,0.025578,0.015473,0.043799,0.015836,0.031317,0.017949,0.005747
2019-01-07,0.029900,-0.002324,0.014595,-0.012750,-0.004960,0.009337,-0.005792,0.019430,-0.002320,0.019490,...,-0.010498,0.059394,0.003853,0.000805,-0.017364,-0.011122,0.040807,0.016563,-0.019647,-0.003636
2019-01-08,0.038710,0.022714,0.014797,0.006077,0.012739,0.017576,0.031605,0.015882,-0.002114,0.014706,...,0.029824,0.089817,0.034033,0.027331,0.000361,0.018917,0.001762,0.015614,0.034943,0.036496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-23,-0.026738,0.008225,0.000903,0.002267,0.002214,-0.008571,0.001075,0.017113,0.005808,0.007145,...,0.000533,0.023293,0.006325,-0.000827,0.005301,0.007313,-0.004110,-0.014011,0.003906,0.001361
2019-12-24,-0.010989,0.008838,-0.002708,-0.003015,-0.008434,0.000000,-0.001790,-0.001923,0.002431,0.002027,...,0.010120,0.004710,0.010266,0.004967,0.008375,0.004537,0.002064,-0.005329,-0.007004,0.002264
2019-12-27,0.013889,-0.008086,0.000000,0.006049,0.007898,0.008646,0.011478,0.005299,-0.003739,-0.005732,...,-0.014237,-0.003906,-0.012858,0.011532,-0.003384,0.000000,0.002471,0.013393,-0.001567,0.009941


In [175]:
class Strategies:
    def __init__(self, Name_strategy, Returns, Risk_free_rate):
        self.Name_strategy = Name_strategy
        self.Returns = Returns
        self.daily_returns = self.Returns.pct_change()
        self.Risk_free_rate = Risk_free_rate
    
    @property
    def annual_return(self):
        return self.daily_returns.mean()
    @property
    def annual_volatility(self):
        return self.daily_returns.std()
    @property
    def Sharpe_Ratio(self):
        return (self.annual_return-self.Risk_free_rate/self.Returns.shape[0]).mean()/self.annual_volatility*np.sqrt(252)
    @property
    def Max_Drawdown(self):
        cumulative_returns = self.Returns
        peak = cumulative_returns.cummax()
        MD = ((cumulative_returns - peak) / peak)
        return {'MDD':MD.min() , 'indx MDD end' : MD.idxmin(), 'indx MDD start': peak.loc[:MD.idxmin()].idxmax()}
    def plot_strategie(self):
        # Create a plot 
        fig, ax1 = plt.subplots(figsize=(8, 5))
        ax1.plot(self.Returns, color='green', label=f'Strategy {self.Name_strategy}')
        ax1.set_title('Strategy Prices Over Time')
        ax1.legend(loc='upper left')
        ax1.grid(True, which='major', axis='x', linestyle='--')
        # Annotating the graph for February 2020 (COVID)
        start_MDD = self.Max_Drawdown['indx MDD start']
        pos_text=self.Returns.mean()+0.05
        ax1.annotate('Start MDD',
             xy=(start_MDD, self.Returns.loc[start_MDD]),
             xytext=(start_MDD, pos_text),
             arrowprops=dict(color='blue', arrowstyle='->'),
             color='blue',
             fontsize=10)
        end_MDD = self.Max_Drawdown['indx MDD end']
        ax1.annotate('End MDD',
             xy=(end_MDD, self.Returns.loc[end_MDD]),
             xytext=(end_MDD, pos_text - 1),
             arrowprops=dict(color='red', arrowstyle='->'),
             color='red',
             fontsize=10)
        fig.text(0.5, 0.1, f'Sharpe Ratio : {self.Sharpe_Ratio}',
        ha='center', fontsize=10, bbox=dict(facecolor='lightgrey', edgecolor='black'))

        # Show the plot
        plt.subplots_adjust(bottom=0.25)
        #plt.tight_layout()
        plt.show()
        

In [176]:
metrics=Strategies("Long only",100*(1+daily_returns["GLE FP Equity"]).cumprod(),0.02)
metrics.plot_strategie()

# Part III: How to build a classical strategy

In [177]:
class CapiWeighted:
    def __init__(self, data: pd.DataFrame, initial_value):
        
        self.data = data
        self.initial_value = initial_value

    def compute_weights(self):
        """Compute allocation for assets at each date (alpha_i)."""
        self.daily_returns = self.data.iloc[:,::2].pct_change()
        daily_market_cap = self.data.iloc[:,1::2]
        total_cap = daily_market_cap.sum(axis=1)

        self.weights = daily_market_cap.div(total_cap, axis=0)

        return self.weights, self.daily_returns
    
    def compute_portfolio_performance(self):
        """Compute portfolio value V_t at each time t."""
        self.compute_weights()
        
        self.daily_returns.iloc[0]=0
        weighted_returns = pd.DataFrame(self.weights.to_numpy() * self.daily_returns.to_numpy(), index=self.weights.index).sum(axis=1)
        weighted_returns.iloc[0] = 0  # pas de rendement le premier jour
        
        V_t = (1+weighted_returns).cumprod() * self.initial_value
        self.performance = V_t
        
        return V_t


In [178]:
Strat_CapiWeighted=CapiWeighted(result[(result.index>='2019-01-01')*(result.index<'2020-01-01')],100)

In [179]:
metrics=Strategies("Capitalization Weighted Portfolio",Strat_CapiWeighted.compute_portfolio_performance(),0.02)
metrics.plot_strategie()